In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
import pandas as pd

CSV_FILE = "tehran_housing.csv"  # اسم فایل دیتاستت

def format_toman(x: int) -> str:
    return f"{int(x):,} تومان"

class HousingGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Tehran Housing Price Estimator")
        self.root.geometry("860x560")
        self.root.minsize(860, 560)

        # ---------- Styles ----------
        self.style = ttk.Style()
        try:
            self.style.theme_use("clam")
        except Exception:
            pass

        self.style.configure("TFrame", background="#0f172a")          # slate-900
        self.style.configure("Card.TFrame", background="#111827")     # gray-900
        self.style.configure("Title.TLabel", background="#0f172a", foreground="white",
                            font=("Segoe UI", 18, "bold"))
        self.style.configure("Sub.TLabel", background="#0f172a", foreground="#cbd5e1",
                            font=("Segoe UI", 10))
        self.style.configure("Label.TLabel", background="#0f172a", foreground="#e2e8f0",
                            font=("Segoe UI", 10, "bold"))
        self.style.configure("Info.TLabel", background="#0f172a", foreground="#94a3b8",
                            font=("Segoe UI", 9))
        self.style.configure("TButton", font=("Segoe UI", 10, "bold"))
        self.style.configure("MetricTitle.TLabel", background="#111827", foreground="#cbd5e1",
                            font=("Segoe UI", 10))
        self.style.configure("MetricValue.TLabel", background="#111827", foreground="white",
                            font=("Segoe UI", 16, "bold"))

        # Treeview style
        self.style.configure("Treeview",
                             font=("Segoe UI", 10),
                             rowheight=26,
                             background="#0b1220",
                             fieldbackground="#0b1220",
                             foreground="#e5e7eb")
        self.style.configure("Treeview.Heading",
                             font=("Segoe UI", 10, "bold"))

        # Root background
        self.root.configure(bg="#0f172a")

        # ---------- Load dataset ----------
        try:
            self.df = pd.read_csv(CSV_FILE)
        except Exception as e:
            messagebox.showerror("Error", f"Could not load dataset:\n{e}")
            root.destroy()
            return

        required = {"neighborhood", "area", "rooms", "age", "parking", "price"}
        if not required.issubset(set(self.df.columns)):
            messagebox.showerror(
                "Error",
                "CSV must contain columns:\n" + ", ".join(sorted(required))
            )
            root.destroy()
            return

        self.neighborhoods = sorted(self.df["neighborhood"].dropna().unique().tolist())

        self.build_ui()

    def build_ui(self):
        # ---------- Layout: two columns ----------
        main = ttk.Frame(self.root, padding=16)
        main.pack(fill="both", expand=True)

        left = ttk.Frame(main)
        right = ttk.Frame(main)
        left.pack(side="left", fill="y", padx=(0, 16))
        right.pack(side="right", fill="both", expand=True)

        # ---------- Header ----------
        header = ttk.Frame(left)
        header.pack(fill="x", pady=(0, 14))

        ttk.Label(header, text="🏠 Tehran Housing Estimator", style="Title.TLabel").pack(anchor="w")
        ttk.Label(header, text="Readable GUI • Similarity filter • Price stats", style="Sub.TLabel").pack(anchor="w", pady=(6, 0))
        ttk.Label(header, text="Rules: area ±10 sqm, age ±5 years", style="Info.TLabel").pack(anchor="w", pady=(6, 0))

        # ---------- Inputs Card ----------
        input_card = ttk.Frame(left, style="Card.TFrame", padding=14)
        input_card.pack(fill="x", pady=(0, 14))

        ttk.Label(input_card, text="Inputs", style="MetricTitle.TLabel").grid(row=0, column=0, columnspan=2, sticky="w", pady=(0, 10))

        ttk.Label(input_card, text="Neighborhood", style="Label.TLabel").grid(row=1, column=0, sticky="w", pady=6)
        self.neighborhood_var = tk.StringVar(value=self.neighborhoods[0] if self.neighborhoods else "")
        self.neighborhood_cb = ttk.Combobox(
            input_card, textvariable=self.neighborhood_var, values=self.neighborhoods,
            state="readonly", width=28
        )
        if self.neighborhoods:
            self.neighborhood_cb.current(0)
        self.neighborhood_cb.grid(row=1, column=1, sticky="w", pady=6)

        ttk.Label(input_card, text="Area (sqm)", style="Label.TLabel").grid(row=2, column=0, sticky="w", pady=6)
        self.area_var = tk.IntVar(value=85)
        self.area_spin = ttk.Spinbox(input_card, from_=40, to=300, textvariable=self.area_var, width=10)
        self.area_spin.grid(row=2, column=1, sticky="w", pady=6)

        ttk.Label(input_card, text="Rooms", style="Label.TLabel").grid(row=3, column=0, sticky="w", pady=6)
        self.rooms_var = tk.IntVar(value=2)
        self.rooms_cb = ttk.Combobox(input_card, textvariable=self.rooms_var, values=[1,2,3,4,5], state="readonly", width=10)
        self.rooms_cb.current(1)
        self.rooms_cb.grid(row=3, column=1, sticky="w", pady=6)

        ttk.Label(input_card, text="Building age", style="Label.TLabel").grid(row=4, column=0, sticky="w", pady=6)
        self.age_var = tk.IntVar(value=8)
        self.age_spin = ttk.Spinbox(input_card, from_=0, to=25, textvariable=self.age_var, width=10)
        self.age_spin.grid(row=4, column=1, sticky="w", pady=6)

        ttk.Label(input_card, text="Parking", style="Label.TLabel").grid(row=5, column=0, sticky="w", pady=6)
        self.parking_var = tk.IntVar(value=1)
        park_frame = ttk.Frame(input_card)
        park_frame.grid(row=5, column=1, sticky="w", pady=6)
        ttk.Radiobutton(park_frame, text="Yes", variable=self.parking_var, value=1).pack(side="left", padx=(0, 10))
        ttk.Radiobutton(park_frame, text="No", variable=self.parking_var, value=0).pack(side="left")

        # Buttons
        btns = ttk.Frame(left)
        btns.pack(fill="x")

        self.estimate_btn = ttk.Button(btns, text="Estimate", command=self.estimate)
        self.estimate_btn.pack(fill="x", pady=(0, 8))

        self.reset_btn = ttk.Button(btns, text="Reset", command=self.reset_inputs)
        self.reset_btn.pack(fill="x")

        # ---------- Right side: Metrics + Table ----------
        top_right = ttk.Frame(right)
        top_right.pack(fill="x")

        # Similar count card
        self.count_card = ttk.Frame(top_right, style="Card.TFrame", padding=14)
        self.count_card.pack(fill="x", pady=(0, 12))
        ttk.Label(self.count_card, text="Similar listings", style="MetricTitle.TLabel").pack(anchor="w")
        self.count_value = ttk.Label(self.count_card, text="—", style="MetricValue.TLabel")
        self.count_value.pack(anchor="w", pady=(6, 0))

        # Metrics row
        metrics_row = ttk.Frame(top_right)
        metrics_row.pack(fill="x", pady=(0, 12))

        self.avg_card = self.make_metric_card(metrics_row, "Average price")
        self.min_card = self.make_metric_card(metrics_row, "Minimum price")
        self.max_card = self.make_metric_card(metrics_row, "Maximum price")

        self.avg_card.pack(side="left", fill="x", expand=True, padx=(0, 10))
        self.min_card.pack(side="left", fill="x", expand=True, padx=(0, 10))
        self.max_card.pack(side="left", fill="x", expand=True)

        # Table of examples
        table_card = ttk.Frame(right, style="Card.TFrame", padding=14)
        table_card.pack(fill="both", expand=True)

        ttk.Label(table_card, text="Sample similar listings (top 20)", style="MetricTitle.TLabel").pack(anchor="w")

        columns = ("area", "rooms", "age", "parking", "price")
        self.tree = ttk.Treeview(table_card, columns=columns, show="headings", height=12)
        self.tree.pack(fill="both", expand=True, pady=(10, 0))

        self.tree.heading("area", text="Area")
        self.tree.heading("rooms", text="Rooms")
        self.tree.heading("age", text="Age")
        self.tree.heading("parking", text="Parking")
        self.tree.heading("price", text="Price")

        self.tree.column("area", width=80, anchor="center")
        self.tree.column("rooms", width=80, anchor="center")
        self.tree.column("age", width=80, anchor="center")
        self.tree.column("parking", width=90, anchor="center")
        self.tree.column("price", width=220, anchor="w")

        # Status label
        self.status = ttk.Label(right, text="Enter inputs and click Estimate.", style="Sub.TLabel")
        self.status.pack(anchor="w", pady=(10, 0))

        # Init metric values
        self.set_metrics(None)

    def make_metric_card(self, parent, title: str):
        card = ttk.Frame(parent, style="Card.TFrame", padding=14)
        ttk.Label(card, text=title, style="MetricTitle.TLabel").pack(anchor="w")
        value = ttk.Label(card, text="—", style="MetricValue.TLabel")
        value.pack(anchor="w", pady=(6, 0))
        card.value_label = value
        return card

    def reset_inputs(self):
        if self.neighborhoods:
            self.neighborhood_cb.current(0)
        self.area_var.set(85)
        self.rooms_var.set(2)
        self.rooms_cb.current(1)
        self.age_var.set(8)
        self.parking_var.set(1)
        self.clear_table()
        self.set_metrics(None)
        self.status.config(text="Reset done. Enter inputs and click Estimate.")

    def clear_table(self):
        for item in self.tree.get_children():
            self.tree.delete(item)

    def set_metrics(self, stats):
        if stats is None:
            self.count_value.config(text="—")
            self.avg_card.value_label.config(text="—")
            self.min_card.value_label.config(text="—")
            self.max_card.value_label.config(text="—")
        else:
            self.count_value.config(text=str(stats["count"]))
            self.avg_card.value_label.config(text=format_toman(stats["avg"]))
            self.min_card.value_label.config(text=format_toman(stats["min"]))
            self.max_card.value_label.config(text=format_toman(stats["max"]))

    def estimate(self):
        neighborhood = self.neighborhood_var.get().strip()

        try:
            area = int(self.area_var.get())
            rooms = int(self.rooms_var.get())
            age = int(self.age_var.get())
            parking = int(self.parking_var.get())
        except Exception:
            messagebox.showerror("Input Error", "Please enter valid numeric values.")
            return

        filtered = self.df[
            (self.df["neighborhood"] == neighborhood) &
            (self.df["rooms"] == rooms) &
            (self.df["parking"] == parking) &
            (self.df["area"].between(area - 10, area + 10)) &
            (self.df["age"].between(age - 5, age + 5))
        ]

        self.clear_table()

        if filtered.empty:
            self.set_metrics({"count": 0, "avg": 0, "min": 0, "max": 0})
            self.avg_card.value_label.config(text="Not found")
            self.min_card.value_label.config(text="Not found")
            self.max_card.value_label.config(text="Not found")
            self.status.config(text="❌ No similar properties found. Try changing area/age or rooms.")
            return

        stats = {
            "count": int(len(filtered)),
            "avg": int(filtered["price"].mean()),
            "min": int(filtered["price"].min()),
            "max": int(filtered["price"].max()),
        }
        self.set_metrics(stats)

        # Show top 20 by closeness (sort by |area-area0| + |age-age0|)
        tmp = filtered.copy()
        tmp["score"] = (tmp["area"] - area).abs() + (tmp["age"] - age).abs()
        tmp = tmp.sort_values(["score", "price"]).head(20)

        for _, row in tmp.iterrows():
            self.tree.insert(
                "", "end",
                values=(
                    int(row["area"]),
                    int(row["rooms"]),
                    int(row["age"]),
                    "Yes" if int(row["parking"]) == 1 else "No",
                    format_toman(int(row["price"]))
                )
            )

        self.status.config(text="✅ Done. Showing statistics + top 20 similar listings.")


if __name__ == "__main__":
    root = tk.Tk()
    app = HousingGUI(root)
    root.mainloop()
